In [1]:
from requests import get
import datetime
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import ipywidgets as widgets
import math
import matplotlib.gridspec as gridspec

In [2]:
def getQuery(query):
    response = get(query, timeout=20)
    if response.status_code >= 400:
        raise RuntimeError("Request failed: %s" % response.text)
    return (response.json()['data'])

def buildQuery(api_address, filters):
    assert (len(filters) != 0)
    F = {'areaType': None,
         'areaName': None,
         'date': None,
         'newCases': None}
    F.update(filters)
    filter_string = ""
    structure_string = '"date": "date",'
    for f, v in F.items():
        if v is not None:
            filter_string += '%s=%s;' % (f, v)
        else:
            if f == 'newCases' and F['areaType'] in ['utla', 'ltla', 'region']:
                structure_string += '"newCases": "newCasesBySpecimenDate",'
            elif f == "newCases":
                structure_string += '"newCases": "newCasesByPublishDate",'
            else:
                structure_string += '"%s": "%s",' % (f, f)
    query = "%sfilters=%s&structure={%s}" % (api_address,
                                            filter_string[:-1],
                                            structure_string[:-1])
    return (query)

def cleanData(results):
    dates = []
    counts = []
    for r in results:
        date = r['date']
        d = datetime.datetime.strptime(date, '%Y-%m-%d')
        dates.append(d)
        counts.append(r['newCases'])

    sw3_counts = [np.median(counts[x:x+3]) for x in np.arange(0, len(counts)-3)]
    sw7_counts = [np.median(counts[x:x+7]) for x in np.arange(0, len(counts)-7)]
    return (dates, counts, sw3_counts, sw7_counts)


def plotData(clean, area_name, area_type, data_to_display):
    daily_colour = '#cfcfcfB3'
    sw3_colour = '#318fb5B3'
    sw7_colour = '#ff4b5cB3'
    dates, counts, sw3_counts, sw7_counts = clean
    f = plt.figure(figsize=(15, 7))
    g = gridspec.GridSpec(5, 5, hspace=4)
    a = f.add_subplot(g[1:4, 0:4])
    handles = []
    labels = []
    if "1" in data_to_display:
        a.bar(dates, counts, color=daily_colour, width=1)
        handles.append(matplotlib.patches.Patch(color=daily_colour))
        labels.append("Daily")
    if "3" in data_to_display:
        a.plot(dates[:-3], sw3_counts, color=sw3_colour, lw=4)
        handles.append(matplotlib.patches.Patch(color=sw3_colour))
        labels.append("3 day median")
    if "7" in data_to_display:
        a.plot(dates[:-7], sw7_counts, color=sw7_colour, lw=4)
        handles.append(matplotlib.patches.Patch(color=sw7_colour))
        labels.append("7 day median")
    if area_type in ['utla', 'ltla', 'region']:
        t ='Specimen'
    else:
        t = "Published"
    a.set_title("Positive SARS-CoV-2 Tests by %s Date\n%s" % (t, area_name), fontsize=14, y=1.1)

    inter = math.ceil(len(dates) / 50)
    a.set_xticks([dates[x] for x in np.arange(0, len(dates), inter)])
    a.set_xticklabels([dates[x].date() for x in np.arange(0, len(dates), inter)], rotation='vertical', fontsize=10)
    a.set_xlim(min(dates), max(dates))
    a.set_ylim(0, max(counts))
    sns.despine()

    leg = f.add_subplot(g[0, 4])
    leg.legend(handles=handles, labels=labels, fontsize=12, loc=(0, 0))
    leg.set_axis_off()

    txt = f.add_subplot(g[1:3, 4])
    txt.text(0, 1.4, 'Daily - Last 7 Days', fontweight='bold')
    y = 1.2
    for i in np.arange(0, 7):
        txt.text(0, y, dates[i].date())
        txt.text(0.6, y, counts[i], ha='left')
        y -= 0.2
    txt.set_ylim(0, 1.4)
    txt.set_axis_off()

    buff = f.add_subplot(g[0, 0:4])
    buff.set_axis_off()
    return (f)

def runAll(area_type, area_name, data_to_display):
    if area_name != "Select Area" and area_name != "-":
        query = buildQuery(api_address, filters={'areaType': area_type, 'areaName': area_name})
        results = getQuery(query)
        clean = cleanData(results)
        f = plotData(clean, area_name, area_type, data_to_display)

def getList(typ):
    if typ != "-":
        datenow = datetime.date.strftime(datetime.date.today() - datetime. timedelta(days=1), "%Y-%m-%d")
        query = buildQuery(api_address, {'areaType': typ, 'date': datenow})
        results = getQuery(query)
        return ([x['areaName'] for x in results])
    else:
        return (["-"])

def changeArea(val):
    if val['new'] != "-":
        selected_area.options = ["Select Area"] + getList(val['new'])
    else:
        selected_area.options = ["-"]

In [7]:
area_types = {"Select Area Type": "-",
              'Local Authority (District)': 'ltla',
              'Local Authority (County)': 'utla',
              'Region': 'region',
              'Nation': 'nation',
              'Total':'overview'}
api_address = 'https://api.coronavirus.data.gov.uk/v1/data?'

# SARS-CoV-2 Data by Date and Area (England)

### Sorry - the API has changed so this needs to be updated, it will be back up and running ASAP.

Select an area of England to see the number of recorded positive SARS-CoV-2 samples in the area for each day since the beginning of the outbreak.

National data is also available for Scotland, Wales and Northern Ireland, plus UK wide totals.

The data automatically update at midnight. Please note in many cases local authority data defaults to zero for the most recent 2-3 days until the latest counts are made available.

Source: [coronavirus.data.gov.uk](http://coronavirus.data.gov.uk)


In [9]:
"""
areas = widgets.Dropdown(options=area_types, description="Area Type")
selected_area = widgets.Dropdown(options=['-'], description='Area')
areas.observe(changeArea, 'value')
opts = {'Daily': "1", '3 Day Median': "3", '7 Day Median': "7"}
data_to_display = widgets.SelectMultiple(options=opts,
                                         value=["1", "3", "7"],
                                         rows=3,
                                         description='Data To Show')
display(widgets.interactive(runAll, area_name=selected_area, area_type=areas, data_to_display=data_to_display))

print ("Last updated %s" % datetime.date.strftime(datetime.date.today() - datetime. timedelta(days=1), "%Y-%m-%d"))
"""

'\nareas = widgets.Dropdown(options=area_types, description="Area Type")\nselected_area = widgets.Dropdown(options=[\'-\'], description=\'Area\')\nareas.observe(changeArea, \'value\')\nopts = {\'Daily\': "1", \'3 Day Median\': "3", \'7 Day Median\': "7"}\ndata_to_display = widgets.SelectMultiple(options=opts,\n                                         value=["1", "3", "7"],\n                                         rows=3,\n                                         description=\'Data To Show\')\ndisplay(widgets.interactive(runAll, area_name=selected_area, area_type=areas, data_to_display=data_to_display))\n\nprint ("Last updated %s" % datetime.date.strftime(datetime.date.today() - datetime. timedelta(days=1), "%Y-%m-%d"))\n'